In [1]:
import lightgbm as lgbm


import sklearn as sk
import pandas as pd
import sklearn.neural_network

import base_functions as funcs
from sklearn.model_selection import train_test_split
from sklearn.ensemble  import RandomForestClassifier
from sklearn import metrics
import numpy as np
import imblearn
from sklearn.linear_model  import LogisticRegression
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score

data, labels = funcs.ReadCSV('./datasets/creditcard.csv', mark='Class')
data = data.drop(columns=['Time'])

In [2]:
split_index=int(len(data)*0.8)
##РАЗБИЕНИЕ ПО ВРЕМЕНИ
data_train, data_test, labels_train, labels_test = (data.iloc[0:split_index], data.iloc[split_index:],
                                                    labels.iloc[0:split_index], labels.iloc[split_index:])


In [3]:
## LightGBM по времени
lgbmClassifier = lgbm.LGBMClassifier(objective='binary', n_estimators=600, metric='prc',
                                     class_weight='balanced',#,reg_lambda=1
                                     learning_rate=0.055,
                                     max_depth=9,
                                     reg_lambda=0.01)

lgbmClassifier.fit(data_train , labels_train)

predicts= lgbmClassifier.predict_proba(data_test )[:, 1]
 

print('average_precision_score+ time: ' , average_precision_score(labels_test, predicts))
print('roc_auc_score: ' , roc_auc_score(labels_test, predicts)) 

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 417, number of negative: 227428
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.055331 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 227845, number of used features: 29
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 

In [4]:
##RandomForest по времени

RandomForestClassifier = RandomForestClassifier(class_weight='balanced', n_estimators=600, n_jobs=-1,max_depth=9)
RandomForestClassifier.fit(data_train , labels_train)

predicts= RandomForestClassifier.predict_proba(data_test )[:, 1]
 

print('average_precision_score+ time: ' , average_precision_score(labels_test, predicts))
print('roc_auc_score: ' , roc_auc_score(labels_test, predicts)) 

average_precision_score+ time:  0.8113981532863804
roc_auc_score:  0.987397940947258


In [5]:
## Логрег по времени

LogisticRegressionClassifier =  LogisticRegression(class_weight='balanced' , max_iter=300)
LogisticRegressionClassifier.fit(data_train , labels_train)

predicts= LogisticRegressionClassifier.predict_proba(data_test )[:, 1]
 

print('average_precision_score+ time: ' , average_precision_score(labels_test, predicts))
print('roc_auc_score: ' , roc_auc_score(labels_test, predicts)) 

average_precision_score+ time:  0.7637059726921287
roc_auc_score:  0.986307826627056


C:\Users\bobor\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [6]:
## ВАЖНО: при запусках MLP результат варьируется от запуска к запуску
## MLP по времени
MLPClassifier=sklearn.neural_network.MLPClassifier(hidden_layer_sizes=(58),
                                                              alpha=0.0001,
                                                              activation='relu',
                                                              early_stopping=True,
                                                              )
MLPClassifier.fit(data_train , labels_train)

predicts= MLPClassifier.predict_proba(data_test )[:, 1]
 

print('average_precision_score+ time: ' , average_precision_score(labels_test, predicts))
print('roc_auc_score: ' , roc_auc_score(labels_test, predicts)) 

average_precision_score+ time:  0.7627341869920368
roc_auc_score:  0.9634276138074898


In [7]:
##LightGBM 5 случайных разбиений датасета
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=42)


scoring = [ 'average_precision','roc_auc']
scores = cross_validate(lgbmClassifier, data, labels, cv=cv, scoring=scoring)
print(scores)

for i in scores.keys():
    print(i+':', sum(scores[i])/ len(scores[i]))

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 394, number of negative: 227451
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010364 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 227845, number of used features: 29
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_

In [8]:
##RandomForestClassifier 5 случайных разбиений датасета
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=42)


scoring = [ 'average_precision','roc_auc']
scores = cross_validate(RandomForestClassifier, data, labels, cv=cv, scoring=scoring)
print(scores)

for i in scores.keys():
    print(i+':', sum(scores[i])/ len(scores[i]))

{'fit_time': array([66.918154  , 67.78796244, 69.57524133, 66.90465522, 70.07005811]), 'score_time': array([0.19364762, 0.18725729, 0.19055486, 0.2106216 , 0.19840932]), 'test_average_precision': array([0.84476218, 0.8423726 , 0.81983552, 0.84503464, 0.77418276]), 'test_roc_auc': array([0.98532733, 0.97921799, 0.99317008, 0.99173349, 0.9890237 ])}
fit_time: 68.25121421813965
score_time: 0.19609813690185546
test_average_precision: 0.8252375388962327
test_roc_auc: 0.9876945166545186


In [9]:
##LogisticRegressionClassifier 5 случайных разбиений датасета
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=42)


scoring = [ 'average_precision','roc_auc']
scores = cross_validate(LogisticRegressionClassifier, data, labels, cv=cv, scoring=scoring)
print(scores)

for i in scores.keys():
    print(i+':', sum(scores[i])/ len(scores[i]))

C:\Users\bobor\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\bobor\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/st

{'fit_time': array([5.34881449, 5.61427164, 5.61347604, 5.15121961, 5.40976977]), 'score_time': array([0.03120685, 0.03132915, 0.03123856, 0.03168726, 0.01560092]), 'test_average_precision': array([0.74876514, 0.77773885, 0.73146762, 0.8452822 , 0.67524442]), 'test_roc_auc': array([0.97954949, 0.97665522, 0.99229776, 0.99163531, 0.98879694])}
fit_time: 5.42751030921936
score_time: 0.028212547302246094
test_average_precision: 0.7556996460320804
test_roc_auc: 0.9857869440362599


C:\Users\bobor\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:
##MLPClassifier 5 случайных разбиений датасета
##РЕЗУЛЬТАТ НА MLP стохастичен! каждый запуск уникален.
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=42)


scoring = [ 'average_precision','roc_auc']
scores = cross_validate(MLPClassifier, data, labels, cv=cv, scoring=scoring)
print(scores)

for i in scores.keys():
    print(i+':', sum(scores[i])/ len(scores[i]))

{'fit_time': array([10.55963826, 11.03337073, 10.89428663, 12.30860257,  9.32882571]), 'score_time': array([0.04640985, 0.04736948, 0.04603004, 0.04432797, 0.03451729]), 'test_average_precision': array([0.83720697, 0.83337803, 0.85126634, 0.85340804, 0.7650489 ]), 'test_roc_auc': array([0.95372238, 0.95104829, 0.94274457, 0.92772904, 0.93358446])}
fit_time: 10.82494478225708
score_time: 0.04373092651367187
test_average_precision: 0.8280616541733595
test_roc_auc: 0.9417657471679775
